In [ ]:
%%bash
ls /sys/bus/usb-serial/devices/ | sed "s/^/\/dev\//g"

In [ ]:
import serial

In [ ]:
ser1 = serial.Serial('/dev/ttyUSB0', 9600)    #     고감도센서
ser2 = serial.Serial('/dev/ttyUSB3', 9600)    #     비밀번호
ser3 = serial.Serial('/dev/ttyUSB4', 9600)    #     진동

In [ ]:
import RPi.GPIO as GPIO 
import time 

#   [ 서보모터 설정 ]
servoPin = 23                          
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPin, GPIO.OUT)
servo = GPIO.PWM(servoPin, 50)         #    servoPin이라는 채널에 50이라는 주파수
servo.start(2.5)                       #    서보모터 초깃값 설정


#   [ 비밀번호 설정 ]
password = [8, 1, 3,  1, 0]            #   키패드 구성 및 설명
                                       #  { '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '*', '#' }
                
                                       #    '*' :  입력 종료
                                       #    '#' :  마지막 입력 취소


#    [ 부저 설정 ]
buzzerPin = 12                         
scale = [ 261, 294, 329, 349, 392, 440, 493, 523, 587, 659, 698, 784 ]   # 12개
#          도   레   미   파   솔   라   시   도   레   미   파   솔 
GPIO.setup(buzzerPin, GPIO.OUT)
buzzer = GPIO.PWM(buzzerPin, 100)
# buzzer.start(100)
buzzer.ChangeDutyCycle(90)
beep = [ 9, 2, 9, 2, 9, 2, 9, 2 ]     #    경보음(삐뽀)   8개
dingdong = [ 0, 2, 4, 7 ]             #     딩동댕        4개
error = [ 6, 6, 6 ]                   #    비밀번호 오류  3개



#    [ 터치 센서 설정]
touch = True

# TODO 최초 서보모터 잠그기    ###############################################



while True:
    try:
        tou = ser1.readline()       #    터치센서
        vib = ser3.readline()       #    진동감지
        pas = ser2.readline()       #    비밀번호


        if int(tou[0]) == 57 and int(vib[0]) == 57:
            print(999)

        if pas[0] == 0:          #    999는 베이스
            print(999)
        else:
            print(int(pas[0]) - 48)

        # 진동 센서가 감지 되거나, 터치 센서가 감지 되거나, 화재경보 센서가 감지 됐을 경우
        if touch:
            if int(tou[0]) == 49 or int(vib[0]) == 49:   # 부저는 자동으로 울림
                print("        Problem Occurred !!!")
                for i in range(8):
                    buzzer.ChangeFrequency(scale[beep[i]])
                    time.sleep(0.3)
                break
        if int(tou[1]) == 49 and int(tou[2]) == 57:
            print("        Fire Occurred !!!")
            break

        # 비밀번호 입력        
        if pas[0] != 0:

            num = int(pas[0]) - 48

            if int(pas[0]) == 42:              #    '*' 입력을 받았을 경우 입력 종료
                buzzer.ChangeFrequency(scale[10])
                if password == test:             #    비밀번호 성공
                    touch = False
                    print("        The Safe Is Opened")
                    for i in range(4):
                        buzzer.ChangeFrequency(scale[dingdong[i]])
                        time.sleep(0.5)
                    servo.ChangeDutyCycle(7.5)     #    서보모터 90도 회전

                else:                              #    비밀번호 실패
                    print("        Wrong !!!")
                    for i in range(3):
                        buzzer.ChangeFrequency(scale[error[i]])
                        time.sleep(0.5)
                    break
            elif 0 <= num and num <= 9:
                num = int(pas[0]) - 48
                buzzer.ChangeFrequency(scale[num])
                print("Num {} has been pressed".format(num))
                test.append(num)
            elif int(pas[0]) == 35:           #    '#' 입력을 받았을 경우
                buzzer.ChangeFrequency(scale[11])
                del test[-1]

    except KeyboardInterrupt:
        break

In [ ]:
# Step1. Touch 센서


import RPi.GPIO as GPIO 
import time 

# #   [ 서보모터 설정 ]
# servoPin = 23                          
# GPIO.setmode(GPIO.BCM)
# GPIO.setup(servoPin, GPIO.OUT)
# servo = GPIO.PWM(servoPin, 50)         #    servoPin이라는 채널에 50이라는 주파수
# servo.start(2.5)                       #    서보모터 초깃값 설정


# #   [ 비밀번호 설정 ]
# password = [8, 1, 3,  1, 0]            #   키패드 구성 및 설명
#                                        #  { '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '*', '#' }
                
#                                        #    '*' :  입력 종료
#                                        #    '#' :  마지막 입력 취소


#    [ 부저 설정 ]
buzzerPin = 12                         
scale = [ 261, 294, 329, 349, 392, 440, 493, 523, 587, 659, 698, 784 ]   # 12개
#          도   레   미   파   솔   라   시   도   레   미   파   솔 
GPIO.setmode(GPIO.BCM)
GPIO.setup(buzzerPin, GPIO.OUT)
buzzer = GPIO.PWM(buzzerPin, 100)
# buzzer.start(100)
buzzer.ChangeDutyCycle(90)
beep = [ 9, 2, 9, 2, 9, 2 ]     #    경보음(삐뽀)   6개
dingdong = [ 0, 2, 4, 7 ]             #     딩동댕        4개
error = [ 6, 6, 6 ]                   #    비밀번호 오류  3개



#    [ 터치 센서 설정]
touch = True

ser1.flushInput()     # 센서 및 부저
while True:
    try:
        tou = ser1.readline()
        print(int(tou[0]))
        
        if touch:
            if int(tou[0]) == 49:   # 부저는 자동으로 울림
                print("        Problem Occurred !!!")
                for i in range(6):
                    buzzer.start(1)
                    buzzer.ChangeFrequency(scale[beep[i]])
                    time.sleep(0.4)
                    buzzer.stop()
                break
        if int(tou[1]) == 49 and int(tou[2]) == 57:
            print("        Fire Occurred !!!")
            for i in range(6):
                buzzer.start(1)
                buzzer.ChangeFrequency(scale[beep[i]])
                time.sleep(0.1)
                buzzer.stop()
            break
        
    except KeyboardInterrupt:
        break



In [ ]:
import RPi.GPIO as GPIO 
import time 
#   [ 서보모터 설정 ]
servoPin = 18                          
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPin, GPIO.OUT)
servo = GPIO.PWM(servoPin, 50)         #    servoPin이라는 채널에 50이라는 주파수
servo.start(1.5)                       #    서보모터 초깃값 설정
servo.ChangeDutyCycle(2.5)
time.sleep(1)
servo.stop()

In [ ]:
# Step2. 비밀번호

import RPi.GPIO as GPIO 
import time 

#   [ 서보모터 설정 ]
servoPin = 18                          
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPin, GPIO.OUT)
servo = GPIO.PWM(servoPin, 50)         #    servoPin이라는 채널에 50이라는 주파수
servo.start(1.5)                       #    서보모터 초깃값 설정
servo.ChangeDutyCycle(7.5)
time.sleep(1)
# servo.stop()

#   [ 비밀번호 설정 ]
# password = [8, 1, 3,  1, 0]            #   키패드 구성 및 설명
#                                        #  { '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '*', '#' }
                
#                                        #    '*' :  입력 종료
#                                        #    '#' :  마지막 입력 취소


#    [ 부저 설정 ]
buzzerPin = 12                         
scale = [ 261, 294, 329, 349, 392, 440, 493, 523, 587, 659, 698, 784 ]   # 12개
#          도   레   미   파   솔   라   시   도   레   미   파   솔 
GPIO.setup(buzzerPin, GPIO.OUT)
buzzer = GPIO.PWM(buzzerPin, 100)
# buzzer.start(100)
buzzer.ChangeDutyCycle(90)
dingdong = [ 2, 4, 7, 9 ]             #     딩동댕        4개
error = [ 6, 6, 6 ]                   #    비밀번호 오류  3개



#    [ 터치 센서 설정]
# touch = True
ser2.flushInput()
test = []
password = [ 0, 1 ]

    
while True:
    try:
        while True:
            try:
                pas = ser2.readline()
                if pas[0] == 0:          #    999는 기본
                        print(" ")
                else:
                    print(int(pas[0]) - 48)

                # 비밀번호 입력        
                if pas[0] == 0:
                    break
                num = int(pas[0]) - 48
                if int(pas[0]) == 42:              #    '*' 입력을 받았을 경우 입력 종료
                    print(test)
                    print(password)
                    buzzer.start(1)
                    buzzer.ChangeFrequency(scale[10])
                    time.sleep(0.3)
                    buzzer.stop()
                    if password == test:          #    비밀번호 성공
                        for i in range(4):
                            buzzer.start(1)
                            buzzer.ChangeFrequency(scale[dingdong[i]])
                            time.sleep(1)
                            buzzer.stop()
                       
                        servo.ChangeDutyCycle(2.5)     #    서보모터 90도 회전
                        time.sleep(1)
                        servo.stop()
                        print(test)
                        print("        The Safe Is Opened")
                        break
                    else:                              #    비밀번호 실패
                        for i in range(3):
                            buzzer.start(1)
                            buzzer.ChangeFrequency(scale[error[i]])
                            time.sleep(0.5)
                            buzzer.stop()
                        print("        Wrong  !!!!")
                        break
                elif 0 <= num and num <= 9:
                    num = int(pas[0]) - 48
                    buzzer.start(1)
                    buzzer.ChangeFrequency(scale[num])
                    time.sleep(0.3)
                    buzzer.stop()
                    print("Num {} has been pressed".format(num))
                    test.append(num)
                elif int(pas[0]) == 35:           #    '#' 입력을 받았을 경우
                    buzzer.start(1)
                    buzzer.ChangeFrequency(scale[11])
                    time.sleep(0.3)
                    buzzer.stop()
                    del test[-1]
            except KeyboardInterrupt:
                break
            
    except KeyboardInterrupt:
        break



In [ ]:
#Step3. 진동 센서

import RPi.GPIO as GPIO 
import time 


#    [ 부저 설정 ]
buzzerPin = 12                         
scale = [ 261, 294, 329, 349, 392, 440, 493, 523, 587, 659, 698, 784 ]   # 12개
#          도   레   미   파   솔   라   시   도   레   미   파   솔 
GPIO.setmode(GPIO.BCM)
GPIO.setup(buzzerPin, GPIO.OUT)
buzzer = GPIO.PWM(buzzerPin, 100)

buzzer.ChangeDutyCycle(90)
beep = [ 9, 2, 9, 2, 9, 2 ]     #    경보음(삐뽀)   8개
dingdong = [ 0, 2, 4, 7 ]             #     딩동댕        4개
error = [ 6, 6, 6 ]                   #    비밀번호 오류  3개

ser3.flushInput() 
vib = ser3.readline()       #    진동감지

while True:
    try:
        if int(vib[0]) == 49:                    # 1
            print("        Vibration Occurred !!!")
            for i in range(6):
                buzzer.start(1)
                buzzer.ChangeFrequency(scale[beep[i]])
                time.sleep(0.3)
                buzzer.stop()
            break
        print(vib[0])

    except KeyboardInterrupt:
        break
   
    
